In [12]:
import pandas as pd
import numpy as np
import os, sys 
sys.path.append(os.environ['HOME'] + '/src/models/')
from deeplearning_models import DLTextClassifier
from feature_based_models import FBConstructivenessClassifier
from sklearn.model_selection import train_test_split

In [13]:
import matplotlib.pyplot as plt
%matplotlib inline

# classifiers / models
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.model_selection import ShuffleSplit
from sklearn.svm import SVC, SVR
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# other
from sklearn.preprocessing import normalize
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
import nltk
import time

import xgboost as xgb
from sklearn.metrics import f1_score, classification_report

In [14]:
def run_dl_experiment(C3_train_df, 
                      C3_test_df, 
                      results_csv_path = os.environ['HOME'] + 'models/predictions.csv',                       
                      model = 'cnn'):


    """    
    """    
    X_train = C3_train_df['pp_comment_text'].astype(str)
    y_train = C3_train_df['constructive_binary']
    
    X_test = C3_test_df['pp_comment_text'].astype(str)
    y_test = C3_test_df['constructive_binary']
    
    dlclf = DLTextClassifier(X_train, y_train)
    
    if model.endswith('lstm'):
        dlclf.build_bilstm()
        
    elif model.endswith('cnn'): 
        dlclf.build_cnn()
        
    dlclf.train(X_train, y_train)
    print('\nTrain results: \n\n')
    dlclf.evaluate(X_train, y_train)
    
    print('\nTest results: \n\n')
    dlclf.evaluate(X_test, y_test)
    results_df = dlclf.write_model_scores_df(C3_test_df, results_csv_path)

### Train on C3 train and test on C3 test

In [15]:
C3_train_df = pd.read_csv(os.environ['C3_TRAIN'])
C3_train_df['pp_comment_text'] = C3_train_df['pp_comment_text'].astype(str)

C3_test_df = pd.read_csv(os.environ['C3_TEST'])
C3_test_df['pp_comment_text'] = C3_test_df['pp_comment_text'].astype(str)

In [16]:
y_C3_train = C3_train_df.constructive_binary     
X_C3_train = C3_train_df.drop(['constructive_binary'], axis = 1)

y_C3_test = C3_test_df.constructive_binary     
X_C3_test = C3_test_df.drop(['constructive_binary'], axis = 1)

In [5]:
classifier = FBConstructivenessClassifier(X_C3_train, y_C3_train, X_C3_test, y_C3_test)

In [6]:
models = {'logistic regression': LogisticRegression, 
          'SVM' : SGDClassifier, 
          'random forest' : RandomForestClassifier, 
          'xgboost' : xgb.XGBClassifier
         }

for model_name, model_class in models.items():
    t = time.time()
    print(model_name, ":")
    m = model_class()
    pipeline = classifier.train_pipeline(classifier = model_class())    
    classifier.show_scores(pipeline)
    elapsed_time = time.time() - t
    print("Elapsed time: %.1f s" % elapsed_time)
    print()

SVM :
Classifier:  SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)
Feature set:  ['ngram_feats', 'tfidf_feats', 'length_feats', 'argumentation_feats', 'text_quality_feats', 'named_entity_feats', 'perspective_content_value_feats', 'perspective_aggressiveness_feats', 'perspecitive_toxicity_feats']
COMMENTS COL:  pp_comment_text


/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Model trained and pickled in file:  /home/vkolhatk/dev/constructiveness/models/saved_model.h5
Training accuracy:   1.00
Validation accuracy: 0.92
TRAIN CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.99      1.00      1.00      4391
        1.0       1.00      1.00      1.00      5209

avg / total       1.00      1.00      1.00      9600

VALIDATION CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.89      0.92      0.91      1093
        1.0       0.94      0.91      0.92      1307

avg / total       0.92      0.92      0.92      2400

sklearn micro-F1-Score: 0.91625
Elapsed time: 42.2 s

xgboost :
Classifier:  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_s

/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Training accuracy:   0.95


...

Validation accuracy: 0.94


/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


TRAIN CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.94      0.95      0.95      4391
        1.0       0.96      0.95      0.96      5209

avg / total       0.95      0.95      0.95      9600



/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


VALIDATION CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.92      0.94      0.93      1093
        1.0       0.95      0.93      0.94      1307

avg / total       0.94      0.94      0.94      2400

sklearn micro-F1-Score: 0.9375
Elapsed time: 102.8 s

logistic regression :
Classifier:  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Feature set:  ['ngram_feats', 'tfidf_feats', 'length_feats', 'argumentation_feats', 'text_quality_feats', 'named_entity_feats', 'perspective_content_value_feats', 'perspective_aggressiveness_feats', 'perspecitive_toxicity_feats']
COMMENTS COL:  pp_comment_text
Model trained and pickled in file:  /home/vkolhatk/dev/constructiveness/models/saved_model.h5
Training accuracy:   1.00
Validation accu

In [7]:
pipeline = classifier.train_pipeline(classifier = xgb.XGBClassifier())    
classifier.show_scores(pipeline)
classifier.write_model_scores_df(pipeline, results_csv_path = os.environ['HOME'] + 'models/feature_based_model_predictions.csv')

Classifier:  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
Feature set:  ['ngram_feats', 'tfidf_feats', 'length_feats', 'argumentation_feats', 'text_quality_feats', 'named_entity_feats', 'perspective_content_value_feats', 'perspective_aggressiveness_feats', 'perspecitive_toxicity_feats']
COMMENTS COL:  pp_comment_text
Model trained and pickled in file:  /home/vkolhatk/dev/constructiveness/models/saved_model.h5


/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Training accuracy:   0.95


/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Validation accuracy: 0.94


...

TRAIN CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.94      0.95      0.95      4391
        1.0       0.96      0.95      0.96      5209

avg / total       0.95      0.95      0.95      9600



/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


VALIDATION CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.92      0.94      0.93      1093
        1.0       0.95      0.93      0.94      1307

avg / total       0.94      0.94      0.94      2400

sklearn micro-F1-Score: 0.9375


/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Predictions file written:  /home/vkolhatk/dev/constructiveness/models/feature_based_model_predictions.csv


In [6]:
def run_experiments_on_feature_sets(fb_classifier,
                                    classifier = xgb.XGBClassifier(),
                                    feature_sets = [['ngram_feats'], 
                                                    ['tfidf_feats'], 
                                                    ['length_feats'], 
                                                    ['argumentation_feats'], 
                                                    ['text_quality_feats'], 
                                                    ['named_entity_feats'], 
                                                    ['perspective_content_value_feats'], 
                                                    ['perspective_aggressiveness_feats'], 
                                                    ['perspecitive_toxicity_feats']]):
    
    all_feats = []
    for feature_set in feature_sets: 
        print('\n\nFeature set:', feature_set)
        all_feats.extend(feature_set)
        pipeline = fb_classifier.train_pipeline(classifier = classifier, 
                                             feature_set = feature_set)    
        fb_classifier.show_scores(pipeline)

    print('\n\nResults with all features: ', all_feats)    
    pipeline = fb_classifier.train_pipeline(classifier = classifier, 
                                         feature_set = all_feats)    
    fb_classifier.show_scores(pipeline)

        

### Train on C3 train and test on NYT

In [6]:
C3_train_df = pd.read_csv(os.environ['C3_TRAIN'])
C3_train_df['pp_comment_text'] = C3_train_df['pp_comment_text'].astype(str)

#NYT_df = pd.read_csv(os.environ['NYT_YNACC'])
NYT_df = pd.read_csv(os.environ['HOME'] + 'data/processed/NYT_YNACC_feats_preprocessed.csv')
NYT_df['pp_comment_text'] = NYT_df['pp_comment_text'].astype(str)

C3_test_df = pd.read_csv(os.environ['C3_TEST'])
C3_test_df['pp_comment_text'] = C3_test_df['pp_comment_text'].astype(str)

In [7]:
y_C3_train = C3_train_df.constructive_binary     
X_C3_train = C3_train_df.drop(['constructive_binary'], axis = 1)

y_NYT = NYT_df.constructive_binary
X_NYT = NYT_df.drop(['constructive_binary'], axis = 1)

y_C3_test = C3_test_df.constructive_binary     
X_C3_test = C3_test_df.drop(['constructive_binary'], axis = 1)

In [29]:
fb_classifier = FBConstructivenessClassifier(X_C3_train, y_C3_train, X_NYT, y_NYT)

In [17]:
run_experiments_on_feature_sets(fb_classifier)



Feature set: ['ngram_feats']
Classifier:  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
Feature set:  ['ngram_feats']
COMMENTS COL:  pp_comment_text
Model trained and pickled in file:  /home/vkolhatk/dev/constructiveness/models/saved_model.h5


/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Training accuracy:   0.83


/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Validation accuracy: 0.75


...

/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Validation accuracy: 0.84


/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


TRAIN CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.94      0.95      0.95      4391
        1.0       0.96      0.95      0.96      5209

avg / total       0.95      0.95      0.95      9600

VALIDATION CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.88      0.77      0.82     15178
        1.0       0.80      0.90      0.84     15147

avg / total       0.84      0.84      0.83     30325

sklearn micro-F1-Score: 0.8350865622423743


/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [18]:
run_experiments_on_feature_sets(fb_classifier, classifier=SGDClassifier())



Feature set: ['ngram_feats']
Classifier:  SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)
Feature set:  ['ngram_feats']
COMMENTS COL:  pp_comment_text


/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Model trained and pickled in file:  /home/vkolhatk/dev/constructiveness/models/saved_model.h5
Training accuracy:   1.00
Validation accuracy: 0.81
TRAIN CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.99      1.00      1.00      4391
        1.0       1.00      0.99      1.00      5209

avg / total       1.00      1.00      1.00      9600

VALIDATION CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.82      0.80      0.81     15178
        1.0       0.81      0.83      0.82     15147

avg / total       0.81      0.81      0.81     30325

sklearn micro-F1-Score: 0.8143775762572134


Feature set: ['tfidf_feats']
Classifier:  SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       s

/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Model trained and pickled in file:  /home/vkolhatk/dev/constructiveness/models/saved_model.h5
Training accuracy:   1.00
Validation accuracy: 0.68
TRAIN CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      4391
        1.0       1.00      1.00      1.00      5209

avg / total       1.00      1.00      1.00      9600

VALIDATION CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.93      0.38      0.54     15178
        1.0       0.61      0.97      0.75     15147

avg / total       0.77      0.68      0.65     30325

sklearn micro-F1-Score: 0.6763066776586975


Feature set: ['length_feats']
Classifier:  SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       

...

TRAIN CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.74      0.62      0.68      4391
        1.0       0.72      0.82      0.77      5209

avg / total       0.73      0.73      0.73      9600

VALIDATION CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.75      0.67      0.70     15178
        1.0       0.70      0.78      0.74     15147

avg / total       0.72      0.72      0.72     30325

sklearn micro-F1-Score: 0.7206265457543282


Feature set: ['perspective_content_value_feats']
Classifier:  SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)
Feature set:  ['perspective_content_value_feats']
COMMENTS COL:  pp_commen

/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureW

TRAIN CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.62      0.35      0.44      4391
        1.0       0.60      0.82      0.69      5209

avg / total       0.61      0.60      0.58      9600

VALIDATION CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.89      0.56      0.69     15178
        1.0       0.68      0.93      0.78     15147

avg / total       0.78      0.75      0.74     30325

sklearn micro-F1-Score: 0.7458532563891178


Feature set: ['perspecitive_toxicity_feats']
Classifier:  SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)
Feature set:  ['perspecitive_toxicity_feats']
COMMENTS COL:  pp_comment_text
M

/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Model trained and pickled in file:  /home/vkolhatk/dev/constructiveness/models/saved_model.h5
Training accuracy:   1.00
Validation accuracy: 0.82
TRAIN CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       1.00      0.99      1.00      4391
        1.0       1.00      1.00      1.00      5209

avg / total       1.00      1.00      1.00      9600

VALIDATION CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.88      0.74      0.80     15178
        1.0       0.78      0.90      0.83     15147

avg / total       0.83      0.82      0.82     30325

sklearn micro-F1-Score: 0.8199175597691674


In [8]:
run_dl_experiment(C3_train_df, NYT_df , model = 'cnn')

len of encoded docs:  9600
Pad sequences (samples x time)
Padded data shape: (9600, 100)
Number of words not found in glove embeddings:  655
Percentage non-zero elements:  0.9757530955461098
Building CNN model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 300)          8116500   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 100, 128)          76928     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 20, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 20, 128)           49280     
_____________________________________________________________

In [9]:
run_dl_experiment(C3_train_df, NYT_df , model = 'lstm')

len of encoded docs:  9600
Pad sequences (samples x time)
Padded data shape: (9600, 100)
Number of words not found in glove embeddings:  655
Percentage non-zero elements:  0.9757530955461098
Building model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 300)          8116500   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               439296    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 8,556,053
Trainable params: 439,553
Non-trainable params: 8,116,500
_________________________________________________________________
None
Training...
Train on 8640 samples, validate 

### Train on C3 train and test on C3 test

In [7]:
fb_classifier = FBConstructivenessClassifier(X_C3_train, y_C3_train, X_C3_test, y_C3_test)

In [8]:
run_experiments_on_feature_sets(fb_classifier, classifier=SGDClassifier())



Feature set: ['ngram_feats']
Classifier:  SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)
Feature set:  ['ngram_feats']
COMMENTS COL:  pp_comment_text


/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Model trained and pickled in file:  /home/vkolhatk/dev/constructiveness/models/saved_model.h5
Training accuracy:   0.99
Validation accuracy: 0.88
TRAIN CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.99      1.00      0.99      4391
        1.0       1.00      0.99      0.99      5209

avg / total       0.99      0.99      0.99      9600

VALIDATION CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.82      0.93      0.87      1093
        1.0       0.93      0.83      0.88      1307

avg / total       0.88      0.88      0.88      2400

sklearn micro-F1-Score: 0.8754166666666666


Feature set: ['tfidf_feats']
Classifier:  SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       s

/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Model trained and pickled in file:  /home/vkolhatk/dev/constructiveness/models/saved_model.h5
Training accuracy:   1.00
Validation accuracy: 0.72
TRAIN CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      4391
        1.0       1.00      1.00      1.00      5209

avg / total       1.00      1.00      1.00      9600

VALIDATION CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.95      0.40      0.56      1093
        1.0       0.66      0.98      0.79      1307

avg / total       0.80      0.72      0.69      2400

sklearn micro-F1-Score: 0.7183333333333334


Feature set: ['length_feats']
Classifier:  SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       

...

TRAIN CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.80      0.94      0.86      4391
        1.0       0.94      0.80      0.86      5209

avg / total       0.88      0.86      0.86      9600

VALIDATION CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.79      0.94      0.86      1093
        1.0       0.94      0.79      0.86      1307

avg / total       0.87      0.86      0.86      2400

sklearn micro-F1-Score: 0.8608333333333333


Feature set: ['named_entity_feats']
Classifier:  SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)
Feature set:  ['named_entity_feats']
COMMENTS COL:  pp_comment_text
Model trained and p

/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureW

VALIDATION CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.74      0.44      0.56      1093
        1.0       0.65      0.87      0.74      1307

avg / total       0.69      0.68      0.66      2400

sklearn micro-F1-Score: 0.6754166666666667


Results with all features:  ['ngram_feats', 'tfidf_feats', 'length_feats', 'argumentation_feats', 'text_quality_feats', 'named_entity_feats', 'perspective_content_value_feats', 'perspective_aggressiveness_feats', 'perspecitive_toxicity_feats']
Classifier:  SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)
Feature set:  ['ngram_feats', 'tfidf_feats', 'length_feats', 'argumentation_feats', 'text_quality_feats', 'named_entity_feat

/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Model trained and pickled in file:  /home/vkolhatk/dev/constructiveness/models/saved_model.h5
Training accuracy:   1.00
Validation accuracy: 0.92
TRAIN CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      4391
        1.0       1.00      1.00      1.00      5209

avg / total       1.00      1.00      1.00      9600

VALIDATION CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.90      0.93      0.91      1093
        1.0       0.94      0.91      0.92      1307

avg / total       0.92      0.92      0.92      2400

sklearn micro-F1-Score: 0.9183333333333333


In [9]:
run_experiments_on_feature_sets(fb_classifier)



Feature set: ['ngram_feats']
Classifier:  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
Feature set:  ['ngram_feats']
COMMENTS COL:  pp_comment_text
Model trained and pickled in file:  /home/vkolhatk/dev/constructiveness/models/saved_model.h5


/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Training accuracy:   0.83


/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Validation accuracy: 0.81


...

/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Validation accuracy: 0.94


/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


TRAIN CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.94      0.95      0.95      4391
        1.0       0.96      0.95      0.96      5209

avg / total       0.95      0.95      0.95      9600

VALIDATION CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.92      0.94      0.93      1093
        1.0       0.95      0.93      0.94      1307

avg / total       0.94      0.94      0.94      2400

sklearn micro-F1-Score: 0.9375


/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### Train on NYT and test on C3 test 

In [ ]:
fb_classifier = FBConstructivenessClassifier(X_NYT, y_NYT, X_C3_test, y_C3_test)

run_experiments_on_feature_sets(fb_classifier, classifier=SGDClassifier())



Feature set: ['ngram_feats']
Classifier:  SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)
Feature set:  ['ngram_feats']
COMMENTS COL:  pp_comment_text


/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Model trained and pickled in file:  /home/vkolhatk/dev/constructiveness/models/saved_model.h5
Training accuracy:   0.99
Validation accuracy: 0.83
TRAIN CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.98      1.00      0.99     15178
        1.0       1.00      0.98      0.99     15147

avg / total       0.99      0.99      0.99     30325

VALIDATION CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.77      0.90      0.83      1093
        1.0       0.91      0.78      0.84      1307

avg / total       0.85      0.83      0.84      2400

sklearn micro-F1-Score: 0.835


Feature set: ['tfidf_feats']
Classifier:  SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, 

/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Model trained and pickled in file:  /home/vkolhatk/dev/constructiveness/models/saved_model.h5
Training accuracy:   0.97
Validation accuracy: 0.72
TRAIN CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.97      0.98      0.97     15178
        1.0       0.98      0.97      0.97     15147

avg / total       0.97      0.97      0.97     30325

VALIDATION CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.88      0.45      0.59      1093
        1.0       0.67      0.95      0.79      1307

avg / total       0.77      0.72      0.70      2400

sklearn micro-F1-Score: 0.7208333333333333


Feature set: ['length_feats']
Classifier:  SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       

...

TRAIN CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.84      0.82      0.83     15178
        1.0       0.83      0.84      0.84     15147

avg / total       0.83      0.83      0.83     30325

VALIDATION CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.80      0.95      0.87      1093
        1.0       0.95      0.80      0.87      1307

avg / total       0.88      0.87      0.87      2400

sklearn micro-F1-Score: 0.87


Feature set: ['named_entity_feats']
Classifier:  SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)
Feature set:  ['named_entity_feats']
COMMENTS COL:  pp_comment_text
Model trained and pickled in file

/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureW

Training accuracy:   0.78
Validation accuracy: 0.61
TRAIN CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.84      0.70      0.77     15178
        1.0       0.74      0.87      0.80     15147

avg / total       0.79      0.78      0.78     30325

VALIDATION CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.58      0.52      0.55      1093
        1.0       0.63      0.69      0.66      1307

avg / total       0.61      0.61      0.61      2400

sklearn micro-F1-Score: 0.61125


Feature set: ['perspecitive_toxicity_feats']
Classifier:  SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)
Feature set:  ['perspecitive_toxicity

/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Model trained and pickled in file:  /home/vkolhatk/dev/constructiveness/models/saved_model.h5
Training accuracy:   1.00
Validation accuracy: 0.78
TRAIN CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     15178
        1.0       1.00      1.00      1.00     15147

avg / total       1.00      1.00      1.00     30325

VALIDATION CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.79      0.70      0.74      1093
        1.0       0.77      0.84      0.80      1307

avg / total       0.78      0.78      0.78      2400

sklearn micro-F1-Score: 0.7770833333333333


In [ ]:
run_experiments_on_feature_sets(fb_classifier)



Feature set: ['ngram_feats']
Classifier:  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
Feature set:  ['ngram_feats']
COMMENTS COL:  pp_comment_text
Model trained and pickled in file:  /home/vkolhatk/dev/constructiveness/models/saved_model.h5


/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Training accuracy:   0.83


/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Validation accuracy: 0.75


...

VALIDATION CLASSIFICATION REPORT

              precision    recall  f1-score   support

        0.0       0.67      0.89      0.76      1093
        1.0       0.87      0.63      0.73      1307

avg / total       0.78      0.75      0.74      2400

sklearn micro-F1-Score: 0.7466666666666667


Feature set: ['tfidf_feats']
Classifier:  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
Feature set:  ['tfidf_feats']
COMMENTS COL:  pp_comment_text
Model trained and pickled in file:  /home/vkolhatk/dev/constructiveness/models/saved_model.h5


/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Training accuracy:   0.84


/home/vkolhatk/anaconda3/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Validation accuracy: 0.64


In [10]:
run_dl_experiment(NYT_df, C3_test_df, model = 'cnn')

len of encoded docs:  30325
Pad sequences (samples x time)
Padded data shape: (30325, 100)
Number of words not found in glove embeddings:  1620
Percentage non-zero elements:  0.9656240059378645
Building CNN model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 100, 300)          14146500  
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 100, 128)          76928     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 20, 128)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 20, 128)           49280     
__________________________________________________________

In [11]:
run_dl_experiment(NYT_df, C3_test_df, model = 'lstm')

len of encoded docs:  30325
Pad sequences (samples x time)
Padded data shape: (30325, 100)
Number of words not found in glove embeddings:  1620
Percentage non-zero elements:  0.9656240059378645
Building model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 300)          14146500  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               439296    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 257       
Total params: 14,586,053
Trainable params: 439,553
Non-trainable params: 14,146,500
_________________________________________________________________
None
Training...
Train on 27292 samples, val

### Train on C3 train and test on NYT

In [ ]:
print clf.coef

In [ ]:
def f_importances(coef, names):
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()


In [ ]:
features_names = ['input1', 'input2']
svm = svm.SVC(kernel='linear')
svm.fit(X, Y)
f_importances(svm.coef_, features_names)